In [1]:
!conda install --yes --file requirements.txt

Channels:
 - conda-forge
Platform: linux-64
Solving environment: done


==> WARNING: A newer version of conda exists. <==
    current version: 25.3.0
    latest version: 25.3.1

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.



# Smart Factory Energy Prediction Challenge

## Goals:
- Analyze the provided sensor data to identify patterns and relationships between environmental factors and equipment energy consumption
- Build a robust regression model to predict equipment energy consumption
- Evaluate the model's performance using appropriate metrics
- Provide actionable insights and recommendations for reducing energy consumption

In [ ]:
# Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score, mean_squared_error

## Data Preprocessing
### Importing the Datasets

In [ ]:
df= pd.read_csv("./data/data.csv", parse_dates=['timestamp'])

### Handling missing data

In [ ]:
print(df.isna().sum())

timestamp                         0
equipment_energy_consumption    844
lighting_energy                 809
zone1_temperature               867
zone1_humidity                  801
zone2_temperature               853
zone2_humidity                  867
zone3_temperature               802
zone3_humidity                  878
zone4_temperature               816
zone4_humidity                  781
zone5_temperature               838
zone5_humidity                  801
zone6_temperature               848
zone6_humidity                  847
zone7_temperature               794
zone7_humidity                  805
zone8_temperature               848
zone8_humidity                  777
zone9_temperature               773
zone9_humidity                  888
outdoor_temperature             806
atmospheric_pressure            842
outdoor_humidity                799
wind_speed                      828
visibility_index                815
dew_point                       826
random_variable1            

#### Data Types

**Time-Series**
- timestamp

**Numeric**
- timestamp
- equipment_energy_consumption
- lighting_energy
- zone1_temperature
- zone1_humidity
- zone2_temperature
- zone2_humidity
- zone3_temperature
- zone3_humidity
- zone4_temperature
- zone4_humidity
- zone5_temperature
- zone5_humidity
- zone6_temperature
- zone6_humidity
- zone7_temperature
- zone7_humidity
- zone8_temperature
- zone8_humidity
- zone9_temperature
- zone9_humidity
- outdoor_temperature
- atmospheric_pressure
- outdoor_humidity
- wind_speed
- visibility_index
- dew_point
- random_variable1
- random_variable2

Since there are no missing values for timestamp filling other columns. Using mean for other numeric data

In [ ]:
def verify_df_missing(df, msg):
    print(msg)
    print(df.isna().sum())

In [ ]:
def fillna(df):
    # timestamp is sorted first for proper forward-filling
    df.sort_values('timestamp', inplace=True)
    # Fill all other numeric columns with their mean
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].fillna(df[numeric_cols].mean())

In [ ]:
# Verify results
fillna(df)
verify_df_missing(df, "Missing values after treatment:")

Missing values after treatment:
timestamp                         0
equipment_energy_consumption    844
lighting_energy                 809
zone1_temperature               867
zone1_humidity                  801
zone2_temperature               853
zone2_humidity                    0
zone3_temperature                 0
zone3_humidity                    0
zone4_temperature                 0
zone4_humidity                    0
zone5_temperature                 0
zone5_humidity                    0
zone6_temperature                 0
zone6_humidity                    0
zone7_temperature                 0
zone7_humidity                    0
zone8_temperature                 0
zone8_humidity                    0
zone9_temperature                 0
zone9_humidity                    0
outdoor_temperature               0
atmospheric_pressure              0
outdoor_humidity                  0
wind_speed                        0
visibility_index                  0
dew_point                       

In [ ]:
num_cols = ['equipment_energy_consumption', 'lighting_energy', 'zone1_temperature', 'zone1_humidity', 'zone2_temperature']

def column_converter(df: pd.DataFrame):
    for col in num_cols:
        if col in df.columns:  # Check if column exists
            df[col] = pd.to_numeric(df[col], errors='coerce')
        else:
            print(f"Warning: Column {col} not found in DataFrame")

column_converter(df)
fillna(df)
verify_df_missing(df, "Missing values after treatment:")

TypeError: column_converter() takes 1 positional argument but 2 were given

In [ ]:
# extract time components
df['hour'] = df['timestamp'].dt.hour
df['day_of_week'] = df['timestamp'].dt.dayofweek
df['month'] = df['timestamp'].dt.month

# Create temperature differential features
for zone in range(1, 10):
    df[f'zone{zone}_temp_diff'] = df[f'zone{zone}_temperature'] - df['outdoor_temperature']

# Calculate average zone temperature and humidity
df['avg_zone_temp'] = df[[f'zone{i}_temperature' for i in range(1, 10)]].mean(axis=1)
df['avg_zone_humidity'] = df[[f'zone{i}_humidity' for i in range(1, 10)]].mean(axis=1)

### Data Description

In [ ]:
print(df.info())
print(df.describe())

Data description shows impossible values in equipment_energy_consumption i.e. negative values are present trying to use abosolute values.

In [ ]:
negative_cols = ['equipment_energy_consumption', 'lighting_energy', 'zone1_humidity', 'zone2_humidity', 'zone3_humidity', 'zone4_humidity', 'zone5_humidity', 'zone6_humidity', 'zone7_humidity', 'zone8_humidity', 'zone9_humidity']
for col in negative_cols:
    df[col] = df[col].abs()

## Exploratory Data Analysis

In [ ]:
cm = sns.light_palette("green", as_cmap=True)
df.head(30).style.background_gradient(cmap=cm)

### Graph 1 - Time Series Plot: Equipment Energy Over Time

- Trends: Long-term increases/decreases in energy usage (e.g., seasonal patterns)
- Cyclical Patterns: Daily/weekly operational cycles visible as repeating peaks and valleys
- Anomalies: Sudden spikes or drops that may indicate equipment issues or measurement errors
- Data Gaps: Missing time periods appear as breaks in the line

Key Questions It Answers:

Are there periods of unusually high/low consumption?

Does energy usage follow predictable patterns over time?

In [ ]:
def energy_consumption_eda(df, timestamp_col='timestamp', energy_col='equipment_energy_consumption'):
 
    sns.set_style("whitegrid")
    plt.rcParams['figure.facecolor'] = 'white'
    
    # Time Series Plot
    def plot_time_series():
        plt.figure(figsize=(15, 6))
        df.set_index(timestamp_col)[energy_col].plot(color='steelblue', linewidth=1)
        plt.title(f'{energy_col.replace("_", " ").title()} Over Time', pad=20)
        plt.ylabel('Energy (Wh)')
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        plt.show()
    
    # Correlation Analysis
    def plot_correlation():
        plt.figure(figsize=(10, len(df.columns)//2))  # Dynamic height
        corr_matrix = df.corr(numeric_only=True)
        sns.heatmap(corr_matrix[[energy_col]].sort_values(
            by=energy_col, ascending=False), 
            annot=True, cmap='coolwarm', center=0,
            annot_kws={'size': 9}, cbar_kws={'shrink': 0.8})
        plt.title(f'Correlation with {energy_col.replace("_", " ").title()}', pad=20)
        plt.tight_layout()
        plt.show()
    
    plot_time_series()
    plot_correlation()

In [ ]:
energy_consumption_eda(df)

### Graph 1

Shows heavy consumption on from midday 2-16 to midday 5-16.

### Graph 2

The correlation graph indicates which features should be used.

## Feature Selection

The purpose of the model is to predict the energy consumption using environmental factors. Thus taking column 'equipment_energy_consumption' as dependent vector y. Excluding lighting_energy as it is a different dependent vector.

In [ ]:
# dp_cols = ['month', 'hour', 'day_of_week', 'lighting_energy', 'random_variable1', 'random_variable2']
# dp_cols = ['lighting_energy', 'visibility_index', 'zone4_humidity', 'zone5_humidity', 'zone7_temperature', 'zone8_temperature', 'zone5_temperature', 'zone6_temp_diff', 'month', 'random_variable1', 'random_variable2']
dp_cols = ['lighting_energy']
df = df.drop(columns=dp_cols).reset_index(drop=True)

In [ ]:
y = df['equipment_energy_consumption']
X = df.drop(columns=['equipment_energy_consumption'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define time feature extractor
def extract_time_features(X):
    df = pd.DataFrame(X, columns=['timestamp'])
    return np.column_stack([
        df['timestamp'].dt.hour,
        df['timestamp'].dt.dayofweek,
        df['timestamp'].dt.month
    ])

## Model Building

k = 15

Model        RMSE        R²
3      RandomForest  162.769622  0.073747
2      DecisionTree  165.966089  0.037010
0  LinearRegression  167.876149  0.014717
1               KNN  171.314964 -0.026062

Best model: RandomForest

k = 30

Model        RMSE        R²
3      RandomForest  161.609187  0.086907
2      DecisionTree  166.718831  0.028255
0  LinearRegression  167.951639  0.013831
1               KNN  170.370777 -0.014783

Best model: RandomForest

k= 20
Model        RMSE        R²
3      RandomForest  161.931802  0.083258
2      DecisionTree  165.929212  0.037438
0  LinearRegression  167.912330  0.014292
1               KNN  170.302471 -0.013970

Best model: RandomForest

In [ ]:
preprocessor = ColumnTransformer([
    ('time_features', FunctionTransformer(extract_time_features), ['timestamp']),
    ('numeric', 'passthrough', X_train.select_dtypes(include=['number']).columns)
])
k = 15

models = {
    'LinearRegression': Pipeline([
        ('preprocessor', preprocessor),
        ('feature_selection', SelectKBest(f_regression, k=k)),
        ('model', LinearRegression())
    ]),
    'KNN': Pipeline([
        ('preprocessor', preprocessor),
        ('feature_selection', SelectKBest(f_regression, k=k)),
        ('model', KNeighborsRegressor(n_neighbors=5))
    ]),
    'DecisionTree': Pipeline([
        ('preprocessor', preprocessor),
        ('feature_selection', SelectKBest(f_regression, k=k)),
        ('model', DecisionTreeRegressor(max_depth=5, random_state=42))
    ]),
    'RandomForest': Pipeline([
        ('preprocessor', preprocessor),
        ('feature_selection', SelectKBest(f_regression, k=k)),
        ('model', RandomForestRegressor(n_estimators=100, random_state=42))
    ])
}

In [ ]:
results = []
for name, pipeline in models.items():
    # Train
    pipeline.fit(X_train, y_train)
    
    # Predict
    y_pred = pipeline.predict(X_test)
    
    # Evaluate
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)
    
    results.append({
        'Model': name,
        'RMSE': rmse,
        'R²': r2
    })

# 5. Results Comparison
results_df = pd.DataFrame(results).sort_values('RMSE')
print(results_df)

# 6. Best model inspection
best_model = models[results_df.iloc[0]['Model']]
print("\nBest model:", results_df.iloc[0]['Model'])

In [ ]:
results_df = results_df.sort_values('RMSE', ascending=False)

# Create figure
plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")

# Bar plot for RMSE
ax = sns.barplot(x='RMSE', y='Model', data=results_df, palette="viridis")
plt.title('Model Performance Comparison (Lower RMSE is Better)', pad=20)
plt.xlabel('Root Mean Squared Error (RMSE)')
plt.ylabel('')

# Add value labels
for p in ax.patches:
    ax.annotate(f"{p.get_width():.1f}", 
               (p.get_width(), p.get_y() + p.get_height()/2),
               ha='left', va='center', xytext=(5, 0), textcoords='offset points')

plt.tight_layout()
plt.show()

## Visualization

results_df = results_df.sort_values('RMSE', ascending=False)

# Create figure
plt.figure(figsize=(10, 6))
sns.set_style("whitegrid")

# Bar plot for RMSE
ax = sns.barplot(x='RMSE', y='Model', data=results_df, palette="viridis")
plt.title('Model Performance Comparison (Lower RMSE is Better)', pad=20)
plt.xlabel('Root Mean Squared Error (RMSE)')
plt.ylabel('')

# Add value labels
for p in ax.patches:
    ax.annotate(f"{p.get_width():.1f}", 
               (p.get_width(), p.get_y() + p.get_height()/2),
               ha='left', va='center', xytext=(5, 0), textcoords='offset points')

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(12, 6))
sns.scatterplot(x='R²', y='RMSE', hue='Model', 
                data=results_df, s=200, palette="viridis")

plt.title('Model Comparison: R² vs RMSE', pad=20)
plt.axvline(x=0, color='grey', linestyle='--')
plt.axhline(y=results_df['RMSE'].mean(), color='grey', linestyle='--')

# Annotate points
for i, row in results_df.iterrows():
    plt.text(x=row['R²']+0.01, y=row['RMSE']+0.5, 
             s=row['Model'], fontsize=10)

plt.tight_layout()
plt.show()